In [1]:
import os
import cv2
import time
import uuid #uuid

In [2]:
IMAGES_PATH ='Data/'

In [3]:
labels = ['0','1']
number_imgs = 15

In [4]:
for label in labels:
    label_path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(label_path):
        os.mkdir(label_path)
    else:
        print(f"Directory '{label_path}' already exists.")
        
    cap = cv2.VideoCapture(1)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH, label, label + '.' + '{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    time.sleep(10) 

2024-02-26 14:45:08.732 Python[5050:142331] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


2024-02-26 14:45:16.154 Python[5050:142331] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 